## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [1]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key



'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'

In [2]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

/Users/moka/Desktop/TARS/myenv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x1085b1880>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x1085ab4c0>, model_name='Gemma2-9b-It', groq_api_key=SecretStr('**********'))

In [4]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi! My name is Poppy and i'm a AI enthusiast")])

AIMessage(content="That's awesome, Poppy!  It's great to meet another AI enthusiast.  \n\nWhat is it about AI that fascinates you the most?  Do you have a favorite area of AI research or application?  I'm always eager to learn more about what excites people about this field. 😊 \n\n", response_metadata={'token_usage': {'completion_tokens': 68, 'prompt_tokens': 22, 'total_tokens': 90, 'completion_time': 0.123636364, 'prompt_time': 0.00016219, 'queue_time': 0.014268590000000001, 'total_time': 0.123798554}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-00a5559b-828a-4e92-a110-1444f84ccb36-0', usage_metadata={'input_tokens': 22, 'output_tokens': 68, 'total_tokens': 90})

In [5]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi! My name is Poppy and i'm a AI enthusiast"),
        AIMessage(content="That's awesome, Poppy!  It's great to meet another AI enthusiast.  \n\nWhat is it about AI that fascinates you the most?  Do you have a favorite area of AI research or application?  I'm always eager to learn more about what excites people about this field. 😊 \n\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You are Poppy, an AI enthusiast!  \n\nThat's what you told me earlier.  Is there anything else you'd like to tell me about yourself? I'm always happy to learn more. 😊  \n", response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 109, 'total_tokens': 157, 'completion_time': 0.087272727, 'prompt_time': 0.003270309, 'queue_time': 0.011250741000000002, 'total_time': 0.090543036}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-84e0daa4-a58b-485f-b241-8e6c654d519c-0', usage_metadata={'input_tokens': 109, 'output_tokens': 48, 'total_tokens': 157})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [6]:
!pip install langchain_community

/Users/moka/.zshenv:1: no such file or directory: /us/libexec/java_home
You should consider upgrading via the '/Users/moka/Desktop/TARS/myenv/bin/python3 -m pip install --upgrade pip' command.


In [7]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [8]:
config={"configurable":{"session_id":"chat1"}}

In [11]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi! My name is Poppy and i'm a AI enthusiast")],
    config=config
)

In [12]:
response.content

"Hi Poppy! That's great to hear.  \n\nWhat sparked your interest in AI? Is there a particular area of AI that you find most fascinating?  \n\nI'm always happy to chat with other AI enthusiasts and learn about their perspectives. 😊  \n\n"

In [13]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Poppy!  You told me earlier. 😊  \n\nIs there anything else I can help you with?\n', response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 167, 'total_tokens': 194, 'completion_time': 0.049090909, 'prompt_time': 0.005183862, 'queue_time': 0.010441337999999998, 'total_time': 0.054274771}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-b78d5d56-8d3d-4611-8f12-bc4cb73bbe63-0', usage_metadata={'input_tokens': 167, 'output_tokens': 27, 'total_tokens': 194})

In [14]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'd be happy to use it! 😊  \n\n"

In [15]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

"Hi John! It's nice to meet you.  \n\nWhat can I do for you today? 😊  \n"

In [16]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'You told me your name is John! 😊  \n\nIs there anything else I can help you with?\n'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [17]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [18]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Poppy")]})

AIMessage(content="Hello Poppy! It's nice to meet you.  \n\nWhat can I do for you today? 😊  \n\n", response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 31, 'total_tokens': 58, 'completion_time': 0.049090909, 'prompt_time': 0.000313679, 'queue_time': 0.014762978999999999, 'total_time': 0.049404588}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-d7a9c74b-1142-4fcc-b987-f756cf607726-0', usage_metadata={'input_tokens': 31, 'output_tokens': 27, 'total_tokens': 58})

In [23]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [19]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Poppy")],
    config=config
)

response

AIMessage(content="Hi Poppy! \n\nIt's nice to meet you.  What can I do for you today? 😊 \n", response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 14, 'total_tokens': 41, 'completion_time': 0.049090909, 'prompt_time': 0.001106666, 'queue_time': 0.012099713000000002, 'total_time': 0.050197575}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-9244cce1-b2e5-430e-8282-2bba88edce5d-0', usage_metadata={'input_tokens': 14, 'output_tokens': 27, 'total_tokens': 41})

In [20]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Poppy!  \n\nI remember that you told me earlier. 😄  Anything else I can help you with? \n'

In [21]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [22]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Poppy")],"language":"Hindi"})
response.content

'नमस्ते Poppy! 😊 \n\nमुझे बहुत अच्छा लगा तुम्हें जानकर। क्या मैं तुम्हारे लिए कुछ कर सकता हूँ?\n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [23]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [24]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Poppy")],"language":"Hindi"},
    config=config
)
repsonse.content

'नमस्ते Poppy! 😊 \n\nमुझे खुशी है कि आप मुझसे बात कर रहे हैं। मैं आपकी मदद करने के लिए तैयार हूँ। \n\nआपसे क्या पूछना चाहेंगे? \n\n'

In [25]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)

In [26]:
response.content

'आपका नाम Poppy है!  😄 \n\n\n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [27]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

/Users/moka/Desktop/TARS/myenv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/moka/Desktop/TARS/myenv/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


[SystemMessage(content="you're a good assistant"),
 HumanMessage(content='I like vanilla ice cream'),
 AIMessage(content='nice'),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

In [28]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"As a helpful assistant, I don't have personal preferences like taste buds! \n\nWhat's your favorite ice cream flavor?  🍦😊  \n\n\n\n\n\n"

In [29]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked: "What is 2 + 2?" \n\nLet me know if you\'d like to try another one! 😊  \n'

In [30]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [31]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As an AI, I don't have access to past conversations or any personal information about you, including your name. If you'd like to tell me your name, I'd be happy to know! 😊  \n"

In [32]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I have no memory of past conversations.  \n\nPlease tell me the math problem you'd like help with! 😊 \n\n"